# Metadata Sync: Materialized Delta Tables (Approach 3)

This notebook materializes Neo4j node labels and relationship types as **managed Delta tables**
in Unity Catalog. When data is written as a Delta table, UC automatically registers the full
schema metadata — column names, types, nullability, row counts, and statistics — making it
browsable in **Catalog Explorer** and queryable via `INFORMATION_SCHEMA`.

**What this proves:** Neo4j graph schema can be synchronized into Unity Catalog with zero
custom API calls. The Spark Connector infers the schema, and `saveAsTable()` does the rest.

### Steps

1. Load configuration from Databricks Secrets
2. Verify Neo4j connectivity
3. Discover all node labels and their properties
4. Create target UC catalog and schema
5. Materialize a single label as a Delta table (test)
6. Verify metadata in `INFORMATION_SCHEMA`
7. Materialize all discovered labels
8. Materialize relationship types
9. Final verification and summary

### Prerequisites

- `neo4j-uc-creds` secret scope configured via `setup.sh`
- Neo4j Spark Connector installed on cluster (`org.neo4j:neo4j-connector-apache-spark_2.12:5.3.10_for_spark_3`)
- Neo4j Python driver installed (`neo4j`)
- **Single user** access mode cluster (required by Spark Connector)

---

## Configuration

In [ ]:
# =============================================================================
# CONFIGURATION — Loaded from Databricks Secrets
# =============================================================================

SCOPE_NAME = "neo4j-uc-creds"

# Neo4j credentials
NEO4J_HOST = dbutils.secrets.get(SCOPE_NAME, "host")
NEO4J_USER = dbutils.secrets.get(SCOPE_NAME, "user")
NEO4J_PASSWORD = dbutils.secrets.get(SCOPE_NAME, "password")

try:
    NEO4J_DATABASE = dbutils.secrets.get(SCOPE_NAME, "database")
except Exception:
    NEO4J_DATABASE = "neo4j"

# Derived URLs
NEO4J_BOLT_URI = f"neo4j+s://{NEO4J_HOST}"

# Target catalog and schema for materialized tables
# Change these to match your environment
TARGET_CATALOG = "neo4j_metadata"
NODES_SCHEMA = "nodes"
RELATIONSHIPS_SCHEMA = "relationships"

print("Configuration loaded from Databricks Secrets:")
print(f"  Secret Scope: {SCOPE_NAME}")
print(f"  Neo4j Host: {NEO4J_HOST}")
print(f"  Bolt URI: {NEO4J_BOLT_URI}")
print(f"  Database: {NEO4J_DATABASE}")
print(f"  Target Catalog: {TARGET_CATALOG}")
print(f"  Nodes Schema: {NODES_SCHEMA}")
print(f"  Relationships Schema: {RELATIONSHIPS_SCHEMA}")

---

## Step 1: Verify Neo4j Connectivity

In [ ]:
# =============================================================================
# VERIFY NEO4J CONNECTIVITY
# =============================================================================

from neo4j import GraphDatabase

print("=" * 60)
print("VERIFY NEO4J CONNECTIVITY")
print("=" * 60)

try:
    driver = GraphDatabase.driver(NEO4J_BOLT_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
    driver.verify_connectivity()
    print("\n[PASS] Driver connectivity verified")

    with driver.session(database=NEO4J_DATABASE) as session:
        result = session.run("RETURN 1 AS test")
        record = result.single()
        print(f"[PASS] Query executed: RETURN 1 = {record['test']}")

        result = session.run("CALL dbms.components() YIELD name, versions RETURN name, versions")
        for record in result:
            print(f"[INFO] Connected to: {record['name']} {record['versions']}")

    driver.close()
    print("\nStatus: PASS")

except Exception as e:
    print(f"\n[FAIL] Connection failed: {e}")
    print("\nStatus: FAIL")

---

## Step 2: Discover Node Labels and Properties

In [ ]:
# =============================================================================
# DISCOVER NEO4J SCHEMA
# =============================================================================
# Uses db.schema.nodeTypeProperties() — built-in, no APOC required.
# Returns label names, property names, property types, and mandatory flags.

from neo4j import GraphDatabase
from collections import defaultdict

print("=" * 60)
print("DISCOVER NEO4J SCHEMA")
print("=" * 60)

discovered_labels = defaultdict(list)
discovered_relationships = defaultdict(list)

try:
    driver = GraphDatabase.driver(NEO4J_BOLT_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

    with driver.session(database=NEO4J_DATABASE) as session:
        # Discover node label properties
        print("\n[INFO] Running CALL db.schema.nodeTypeProperties()...")
        result = session.run("CALL db.schema.nodeTypeProperties()")
        for record in result:
            labels = record["nodeLabels"]
            if len(labels) == 1:
                label = labels[0]
                discovered_labels[label].append({
                    "name": record["propertyName"],
                    "types": record["propertyTypes"],
                    "mandatory": record["mandatory"]
                })

        # Discover relationship type properties
        print("[INFO] Running CALL db.schema.relTypeProperties()...")
        result = session.run("CALL db.schema.relTypeProperties()")
        for record in result:
            rel_type = record["relType"].strip(":`")
            discovered_relationships[rel_type].append({
                "name": record["propertyName"],
                "types": record["propertyTypes"],
                "mandatory": record["mandatory"]
            })

    driver.close()

    # Display discovered schema
    print(f"\nNODE LABELS ({len(discovered_labels)} discovered):")
    print("-" * 50)
    for label, props in sorted(discovered_labels.items()):
        print(f"  {label}: {len(props)} properties")
        for p in props[:5]:
            types_str = ", ".join(p["types"])
            mandatory_str = " [mandatory]" if p["mandatory"] else ""
            print(f"    - {p['name']}: {types_str}{mandatory_str}")
        if len(props) > 5:
            print(f"    ... and {len(props) - 5} more")

    print(f"\nRELATIONSHIP TYPES ({len(discovered_relationships)} discovered):")
    print("-" * 50)
    for rel_type, props in sorted(discovered_relationships.items()):
        prop_count = len([p for p in props if p["name"] is not None])
        print(f"  {rel_type}: {prop_count} properties")

    print(f"\n[PASS] Schema discovery complete")

except Exception as e:
    print(f"\n[FAIL] Schema discovery failed: {e}")
    import traceback
    traceback.print_exc()

---

## Step 3: Create Target Catalog and Schema

In [ ]:
# =============================================================================
# CREATE TARGET CATALOG AND SCHEMAS
# =============================================================================

print("=" * 60)
print("CREATE TARGET CATALOG AND SCHEMAS")
print("=" * 60)

try:
    spark.sql(f"CREATE CATALOG IF NOT EXISTS `{TARGET_CATALOG}`")
    print(f"\n[PASS] Catalog '{TARGET_CATALOG}' exists")
except Exception as e:
    print(f"\n[FAIL] Could not create catalog: {e}")
    print("[INFO] You may need CREATE CATALOG privilege. Ask your admin.")

try:
    spark.sql(f"CREATE SCHEMA IF NOT EXISTS `{TARGET_CATALOG}`.`{NODES_SCHEMA}`")
    print(f"[PASS] Schema '{TARGET_CATALOG}.{NODES_SCHEMA}' exists")
except Exception as e:
    print(f"[FAIL] Could not create schema: {e}")

try:
    spark.sql(f"CREATE SCHEMA IF NOT EXISTS `{TARGET_CATALOG}`.`{RELATIONSHIPS_SCHEMA}`")
    print(f"[PASS] Schema '{TARGET_CATALOG}.{RELATIONSHIPS_SCHEMA}' exists")
except Exception as e:
    print(f"[FAIL] Could not create schema: {e}")

# Verify
print(f"\n[INFO] Target structure:")
print(f"  {TARGET_CATALOG}")
print(f"  ├── {NODES_SCHEMA}")
print(f"  └── {RELATIONSHIPS_SCHEMA}")

---

## Step 4: Materialize One Label (Single Label Test)

Read the first discovered label from Neo4j via the Spark Connector and write it as a
managed Delta table. This validates the full pipeline before running it for all labels.

In [ ]:
# =============================================================================
# MATERIALIZE ONE LABEL AS A DELTA TABLE
# =============================================================================

print("=" * 60)
print("MATERIALIZE SINGLE LABEL (TEST)")
print("=" * 60)

# Pick the first discovered label
test_label = sorted(discovered_labels.keys())[0] if discovered_labels else "Aircraft"
table_name = test_label.lower()
full_table = f"`{TARGET_CATALOG}`.`{NODES_SCHEMA}`.`{table_name}`"

print(f"\n[INFO] Label: {test_label}")
print(f"[INFO] Target table: {full_table}")

try:
    # Read from Neo4j via Spark Connector
    df = spark.read.format("org.neo4j.spark.DataSource") \
        .option("url", NEO4J_BOLT_URI) \
        .option("authentication.type", "basic") \
        .option("authentication.basic.username", NEO4J_USER) \
        .option("authentication.basic.password", NEO4J_PASSWORD) \
        .option("database", NEO4J_DATABASE) \
        .option("labels", f":{test_label}") \
        .load()

    print(f"\n[INFO] Inferred schema for :{test_label}:")
    df.printSchema()

    print(f"[INFO] Sample data (5 rows):")
    df.show(5, truncate=False)

    row_count = df.count()
    col_count = len(df.columns)

    # Write as managed Delta table
    df.write.mode("overwrite").saveAsTable(full_table)

    print(f"\n[PASS] Materialized :{test_label} → {full_table}")
    print(f"  Rows: {row_count}")
    print(f"  Columns: {col_count}")

except Exception as e:
    print(f"\n[FAIL] Materialization failed: {e}")
    import traceback
    traceback.print_exc()

---

## Step 5: Verify Metadata in INFORMATION_SCHEMA

Confirm that the materialized table and its columns are now visible in Unity Catalog's
`INFORMATION_SCHEMA`. This is the key proof that metadata sync worked — the table appears
in Catalog Explorer with full column definitions.

In [ ]:
# =============================================================================
# VERIFY METADATA IN INFORMATION_SCHEMA
# =============================================================================

print("=" * 60)
print("VERIFY METADATA IN INFORMATION_SCHEMA")
print("=" * 60)

# Check tables
print(f"\n[INFO] Tables in {TARGET_CATALOG}.{NODES_SCHEMA}:")
spark.sql(f"""
    SELECT table_name, table_type, comment, created
    FROM `{TARGET_CATALOG}`.information_schema.tables
    WHERE table_schema = '{NODES_SCHEMA}'
    ORDER BY table_name
""").show(truncate=False)

# Check columns for the test table
print(f"\n[INFO] Columns in {TARGET_CATALOG}.{NODES_SCHEMA}.{table_name}:")
spark.sql(f"""
    SELECT ordinal_position, column_name, data_type, is_nullable
    FROM `{TARGET_CATALOG}`.information_schema.columns
    WHERE table_schema = '{NODES_SCHEMA}'
      AND table_name = '{table_name}'
    ORDER BY ordinal_position
""").show(truncate=False)

# Verify row count via SQL
count_result = spark.sql(f"SELECT COUNT(*) AS cnt FROM {full_table}").collect()[0]["cnt"]
print(f"[PASS] Table {full_table} has {count_result} rows")
print(f"[PASS] Metadata is visible in INFORMATION_SCHEMA and Catalog Explorer")

---

## Step 6: Materialize All Discovered Labels

Loop through all discovered node labels and materialize each one as a managed Delta table.

In [ ]:
# =============================================================================
# MATERIALIZE ALL NODE LABELS
# =============================================================================

import time

print("=" * 60)
print(f"MATERIALIZE ALL NODE LABELS ({len(discovered_labels)} labels)")
print("=" * 60)

label_results = []

for label in sorted(discovered_labels.keys()):
    tbl_name = label.lower()
    full_tbl = f"`{TARGET_CATALOG}`.`{NODES_SCHEMA}`.`{tbl_name}`"

    try:
        start = time.time()

        df = spark.read.format("org.neo4j.spark.DataSource") \
            .option("url", NEO4J_BOLT_URI) \
            .option("authentication.type", "basic") \
            .option("authentication.basic.username", NEO4J_USER) \
            .option("authentication.basic.password", NEO4J_PASSWORD) \
            .option("database", NEO4J_DATABASE) \
            .option("labels", f":{label}") \
            .load()

        row_count = df.count()
        col_count = len(df.columns)

        df.write.mode("overwrite").saveAsTable(full_tbl)

        elapsed = time.time() - start
        label_results.append({
            "label": label,
            "table": tbl_name,
            "rows": row_count,
            "columns": col_count,
            "time_s": round(elapsed, 1),
            "status": "PASS"
        })
        print(f"  [PASS] :{label} → {full_tbl} ({row_count} rows, {col_count} cols, {elapsed:.1f}s)")

    except Exception as e:
        elapsed = time.time() - start
        error_msg = str(e).split('\n')[0][:80]
        label_results.append({
            "label": label,
            "table": tbl_name,
            "rows": 0,
            "columns": 0,
            "time_s": round(elapsed, 1),
            "status": f"FAIL: {error_msg}"
        })
        print(f"  [FAIL] :{label} — {error_msg}")

# Summary
passed = [r for r in label_results if r["status"] == "PASS"]
failed = [r for r in label_results if r["status"] != "PASS"]
total_rows = sum(r["rows"] for r in passed)
total_cols = sum(r["columns"] for r in passed)

print(f"\n" + "=" * 60)
print(f"NODE LABELS SUMMARY")
print(f"  Passed: {len(passed)}/{len(label_results)}")
print(f"  Total rows: {total_rows:,}")
print(f"  Total columns: {total_cols}")
if failed:
    print(f"  Failed: {', '.join(r['label'] for r in failed)}")

---

## Step 7: Materialize Relationship Types

Read relationship types from Neo4j and write them as Delta tables in the `relationships` schema.
The Spark Connector's `relationship` option reads relationship data including source/target node IDs.

In [ ]:
# =============================================================================
# MATERIALIZE RELATIONSHIP TYPES
# =============================================================================

import time
from neo4j import GraphDatabase

print("=" * 60)
print("MATERIALIZE RELATIONSHIP TYPES")
print("=" * 60)

# First, discover relationship patterns with source/target labels
rel_patterns = []
try:
    driver = GraphDatabase.driver(NEO4J_BOLT_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
    with driver.session(database=NEO4J_DATABASE) as session:
        result = session.run("""
            CALL db.schema.visualization() YIELD nodes, relationships
            UNWIND relationships AS rel
            RETURN DISTINCT type(rel) AS rel_type,
                   labels(startNode(rel))[0] AS source_label,
                   labels(endNode(rel))[0] AS target_label
        """)
        for record in result:
            rel_patterns.append({
                "type": record["rel_type"],
                "source": record["source_label"],
                "target": record["target_label"]
            })
    driver.close()
    print(f"\n[INFO] Found {len(rel_patterns)} relationship patterns")
except Exception as e:
    print(f"\n[WARN] Could not discover relationship patterns: {e}")
    print("[INFO] Falling back to discovered_relationships keys")
    for rel_type in discovered_relationships:
        rel_patterns.append({"type": rel_type, "source": "*", "target": "*"})

rel_results = []

for pattern in rel_patterns:
    rel_type = pattern["type"]
    source_label = pattern["source"]
    target_label = pattern["target"]
    tbl_name = rel_type.lower()
    full_tbl = f"`{TARGET_CATALOG}`.`{RELATIONSHIPS_SCHEMA}`.`{tbl_name}`"

    try:
        start = time.time()

        reader = spark.read.format("org.neo4j.spark.DataSource") \
            .option("url", NEO4J_BOLT_URI) \
            .option("authentication.type", "basic") \
            .option("authentication.basic.username", NEO4J_USER) \
            .option("authentication.basic.password", NEO4J_PASSWORD) \
            .option("database", NEO4J_DATABASE) \
            .option("relationship", rel_type)

        if source_label and source_label != "*":
            reader = reader.option("relationship.source.labels", f":{source_label}")
        if target_label and target_label != "*":
            reader = reader.option("relationship.target.labels", f":{target_label}")

        df = reader.load()

        row_count = df.count()
        col_count = len(df.columns)

        df.write.mode("overwrite").saveAsTable(full_tbl)

        elapsed = time.time() - start
        rel_results.append({
            "type": rel_type,
            "pattern": f"(:{source_label})-[:{rel_type}]->(:{target_label})",
            "table": tbl_name,
            "rows": row_count,
            "columns": col_count,
            "time_s": round(elapsed, 1),
            "status": "PASS"
        })
        print(f"  [PASS] [:{rel_type}] → {full_tbl} ({row_count} rows, {col_count} cols, {elapsed:.1f}s)")

    except Exception as e:
        elapsed = time.time() - start
        error_msg = str(e).split('\n')[0][:80]
        rel_results.append({
            "type": rel_type,
            "pattern": f"(:{source_label})-[:{rel_type}]->(:{target_label})",
            "table": tbl_name,
            "rows": 0,
            "columns": 0,
            "time_s": round(elapsed, 1),
            "status": f"FAIL: {error_msg}"
        })
        print(f"  [FAIL] [:{rel_type}] — {error_msg}")

# Summary
passed_rels = [r for r in rel_results if r["status"] == "PASS"]
failed_rels = [r for r in rel_results if r["status"] != "PASS"]

print(f"\n" + "=" * 60)
print(f"RELATIONSHIP TYPES SUMMARY")
print(f"  Passed: {len(passed_rels)}/{len(rel_results)}")
print(f"  Total rows: {sum(r['rows'] for r in passed_rels):,}")
if failed_rels:
    print(f"  Failed: {', '.join(r['type'] for r in failed_rels)}")

---

## Step 8: Final Verification and Summary

Query `INFORMATION_SCHEMA` to show all tables created across both schemas. This is the
final proof that Neo4j metadata is synchronized into Unity Catalog.

In [ ]:
# =============================================================================
# FINAL VERIFICATION AND SUMMARY
# =============================================================================

print("=" * 60)
print("FINAL VERIFICATION")
print("=" * 60)

# All tables in the catalog
print(f"\n[INFO] All tables in {TARGET_CATALOG}:")
all_tables_df = spark.sql(f"""
    SELECT table_schema, table_name, table_type
    FROM `{TARGET_CATALOG}`.information_schema.tables
    WHERE table_schema IN ('{NODES_SCHEMA}', '{RELATIONSHIPS_SCHEMA}')
    ORDER BY table_schema, table_name
""")
all_tables_df.show(50, truncate=False)

# Column counts per table
print(f"\n[INFO] Column counts per table:")
spark.sql(f"""
    SELECT table_schema, table_name, COUNT(*) AS column_count
    FROM `{TARGET_CATALOG}`.information_schema.columns
    WHERE table_schema IN ('{NODES_SCHEMA}', '{RELATIONSHIPS_SCHEMA}')
    GROUP BY table_schema, table_name
    ORDER BY table_schema, table_name
""").show(50, truncate=False)

# Overall summary
total_tables = all_tables_df.count()
node_tables = len(passed) if 'passed' in dir() else 0
rel_tables = len(passed_rels) if 'passed_rels' in dir() else 0
total_data_rows = (sum(r['rows'] for r in label_results if r['status'] == 'PASS') +
                   sum(r['rows'] for r in rel_results if r['status'] == 'PASS'))

print("\n" + "=" * 60)
print("METADATA SYNC SUMMARY")
print("=" * 60)
print(f"  Target Catalog: {TARGET_CATALOG}")
print(f"  Node label tables: {node_tables} (in {NODES_SCHEMA})")
print(f"  Relationship tables: {rel_tables} (in {RELATIONSHIPS_SCHEMA})")
print(f"  Total tables: {total_tables}")
print(f"  Total data rows: {total_data_rows:,}")
print(f"\n  All tables are:")
print(f"    - Browsable in Catalog Explorer")
print(f"    - Visible in INFORMATION_SCHEMA")
print(f"    - Queryable via standard SQL")
print(f"    - Governed by UC permissions")
print(f"\n[PASS] Metadata synchronization complete")